In [46]:
import json
import pandas as pd
from copy import deepcopy

In [47]:
%%capture --no-stderr
%pip install -U langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph nomic[local]

## Load Data

In [48]:
# load data

try:
    with  open('data/mails.json', 'r') as f:
        email_database = json.load(f)
except FileNotFoundError:
    raise FileNotFoundError('mails.json not found')

print(f"Il y a {len(email_database)} mails (question+réponse) dans la base de données")

try :
    actis = pd.read_csv('data/actis.csv')
except FileNotFoundError:
    raise FileNotFoundError('actis.csv not found')

print(f"Il y a {len(actis)} actis dans la base de données")

try :
    chambres = pd.read_csv('data/chambres.csv')
except FileNotFoundError:
    raise FileNotFoundError('chambres.csv not found')

print(f"Il y a {len(chambres)} semaines de dispos renseignées pour les chambres dans la base de données")

try :
    accueil = pd.read_csv('data/accueil.csv')
except FileNotFoundError:
    raise FileNotFoundError('accueil.csv not found')

print(f"Il y a {len(accueil)} informations à propos de l'accueil dans la base de données")

Il y a 49 mails (question+réponse) dans la base de données
Il y a 30 actis dans la base de données
Il y a 52 semaines de dispos renseignées pour les chambres dans la base de données
Il y a 7 informations à propos de l'accueil dans la base de données


In [49]:
local_llm = "llama3.1"
local_embedder = "nomic-embed-text"

In [50]:
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_community.chat_models import ChatOllama

embedder = OllamaEmbeddings(model = local_embedder)
llm = ChatOllama(model = local_llm, temperature=0)

## Choose where to fecth the information from

In [51]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

In [52]:
from ollama_instructor.ollama_instructor_client import OllamaInstructorClient

In [53]:
from datetime import datetime

In [54]:
#2023-02-19
datetime.strptime("2023-02-19", "%Y-%m-%d")

datetime.datetime(2023, 2, 19, 0, 0)

In [55]:
## gérer les mails

def try_content_from_mail(mail):
    from langchain_core.documents import Document
    with  open('data/mails.json', 'r') as f:
            email_database = json.load(f)

    full_text = []
    for i, page in enumerate(email_database):
        question = page['question']
        answer = page['answer']
        document = Document(
            page_content=f" QUESTION:\n {question}\n\n ANSWER:\n {answer}",
        )
        full_text.append(document)

    from langchain_chroma import Chroma
    embedder = OllamaEmbeddings(model = local_embedder)

    vectorstore = Chroma()
    vectorstore.delete_collection()
    vectorstore = Chroma.from_documents(documents=full_text, 
                                        embedding=embedder)
    retriever = vectorstore.as_retriever()

    TEMPLATE = """Tu es un assistant qui va m'aider à répondre à des mails. J'ai déjà une base énorme de mails avec des questons et de réponses, et je veux que tu m'aides à répondre à des questions.
    Je vaius te donner en contexte une paire question/réponse parmi mes mails, puis une question. Tu devras me donner la réponse qui correspond le mieux à la question. Tu peux dire "je ne sais pas" si tu ne sais pas.

    Voici un exemple de question et de réponse :
    {context}

    Voici la question à laquelle tu dois répondre:
    {question}

    Tu vas UNIQUEMENT me donner la réponse à donner à la question. Tu n'as pas besoin de me donner la question, je la connais déjà.
    """

    prompt = ChatPromptTemplate.from_template(TEMPLATE)
    rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
    )

    res = rag_chain.invoke(mail)
    return res


In [56]:
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["disponibilite_chambres", "ouverture_accueil", "activite", "other"] = Field(
        ...,
        description="Given a user question choose which datasource would be most relevant for answering their question. disponibilite_chambres refers to room availability, ouverture_accueil refers to the opening hours of the reception, activites refers to the prices of activities, other refers to a question that does not fit into any of the other categories.",
    )

class RoomDates(BaseModel):
    """Gives the start and end dates of a room reservation that a user is asking about, in 2023, as well as the type of room."""

    start_date: str = Field(
        ...,
        description="The start date of the room reservation that the user is asking about.",
    )
    end_date: str = Field(
        ...,
        description="The end date of the room reservation that the user is asking about.",
    )
    room_type: Literal["double", "family", "not_specified"] = Field(
        ...,
        description="The type of room that the user is asking about."
    )

client = OllamaInstructorClient()

def dispo_chambre(mail: str) -> RoomDates:
    response = client.chat_completion(
        model='llama3.1', 
        pydantic_model=RoomDates, 
        messages=[
            {
                'role': 'user',
                'content': mail
            }
        ]
    )
    start_date = response["message"]["content"]["start_date"]
    end_date = response["message"]["content"]["end_date"]
    room_type = response["message"]["content"]["room_type"]

    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")

    data = pd.read_csv('data/chambres.csv')

    week_beg = None
    week_end = None

    for index, row in data.iterrows():
        if datetime.strptime(row['Date_Debut'], "%Y-%m-%d") >= start_date and week_beg is None:
            week_beg = index - 1
        if datetime.strptime(row['Date_Fin'], "%Y-%m-%d") >= end_date:
            week_end = index
        if week_beg is not None and week_end is not None:
            break    
    
    if week_beg is None or week_end is None:
        raise Exception("No data found for this period")

    data = data.iloc[week_beg:week_end]
    return {"start_date": start_date, "end_date": end_date, "room_type": room_type, "data": data}

def choose_route(mail: str) -> RouteQuery:
    response = client.chat_completion(
        model='llama3.1', 
        pydantic_model=RouteQuery, 
        messages=[
            {
                'role': 'user',
                'content': mail
            }
        ]
    )
    return {"question": mail, "datasource": response["message"]["content"]['datasource']}

def continue_root(state):
    question = state["question"]
    datasource = state["datasource"]
    print('-----------------')
    print(f"Question : {question}")
    print(f"Datasource : {datasource}")
    print('-----------------')
    if datasource == "ouverture_accueil":
        data = pd.read_csv('data/accueil.csv')
        PROMPT = f"""Tu es un assistant va répondre à une question de l'utilisateur à propos des horaires d'ouverture de l'accueil. Tu vas être très simple et factuel. Tu vas t'exprimer clairement.

        Question : {question}

        
        Tu vas répondre à l'aide de ces données:
        Données :
        """ + data.to_string()
        return PROMPT
    
    elif datasource == "disponibilite_chambres":
        out = dispo_chambre(question)
        PROMPT = f"""Tu es un assistant va répondre à une question de l'utilisateur à propos de la disponibilité des chambres. Tu vas être très simple et factuel. Tu vas t'exprimer clairement.

        Question : {question}

        Tu vas prendre en compte la période suivante : {out["start_date"]} - {out["end_date"]} pour une chambre de type {out["room_type"]}.

        Tu vas répondre à l'aide de ces données:
        Données :
        """ + out["data"].to_string()
        return PROMPT
    
    elif datasource == "activite":
        data = pd.read_csv('data/actis.csv')
        PROMPT = f"""Tu es un assistant va répondre à une question de l'utilisateur à propos des prix des activités. Tu vas être très simple et factuel. Tu vas t'exprimer clairement.

        Question : {question}

        Tu vas répondre à l'aide de ces données.
        Données :
        """ + data.to_string()
        return PROMPT
    
    elif datasource == "other":
        return try_content_from_mail(question)
        

chain = RunnableLambda(choose_route) | RunnableLambda(continue_root) | llm | StrOutputParser()

In [57]:
#mail = "Je veux venir mardi à 13h à l'accueil, est-ce que c'est possible ?"
#print(chain.invoke(mail))

In [58]:
#mail = "Je veux venir le 15 aout avec ma famille pour une semaine, est-ce que vous avez des chambres disponibles ?"
#print(chain.invoke(mail))

In [59]:
#mail = "Est-ce que vous avez une activitée de soin du visage ?"
#print(chain.invoke(mail))

In [60]:
mail = "Avez-vous une piscine ?"
print(chain.invoke(mail))

-----------------
Question : Avez-vous une piscine ?
Datasource : other
-----------------
Oui, nous avons une piscine sur place.
